# RTPA-Core

Demonstrates the usage of the `PDCBuffer` class from the `rtpa` module to stream and
process IEEE C37.118 synchrophasor data from a Phasor Data Concentrator (PDC) server.

This script connects to a PDC server, streams data, retrieves it as a PyArrow RecordBatch,
converts it to Pandas and Polars DataFrames, and analyzes raw samples and channel locations.
It measures performance metrics (e.g., data retrieval, conversion times, memory usage) and
is suitable for power system monitoring applications.

**Key Features:**
- Connects to a PDC server using IEEE C37.118-2011 (version "v1").
- Streams synchrophasor data and processes it into timeseries DataFrames.
- Retrieves raw data frames and channel metadata for low-level analysis.
- Demonstrates integration with PyArrow, Pandas, and Polars for data processing.

**Usage:**
    Run this script with a running PDC server at the specified IP and port (e.g., 127.0.0.1:8123).
    Ensure the `rtpa` package is installed and the server supports IEEE C37.118.

**Copyright and Authorship:**
    Copyright (c) 2025 Alliance for Sustainable Energy, LLC.
    Developed by Micah Webb at the National Renewable Energy Laboratory (NREL).
    Licensed under the BSD 3-Clause License. See the `LICENSE` file for details.



## Start the Mock PDC or openPDC

I use the mock pdc provided by rtpa-core to demonstrate the pmu filtering capabilities.

In a terminal inside the RTPA-core repository, run the following shell command.

`cargo run mock-pdc --num-pmus=10`

You can also use the openPDC implementation but you will need to change the port number.

In [ ]:
from rtpa import PDCBuffer
import pandas as pd
import polars as pl
import pyarrow
from time import sleep, time
import binascii  # For hex conversion

# Initialize the PDCBuffer instance
pdc_buffer = PDCBuffer()

# Connect to the PDC server at 127.0.0.1:8123 with ID code 235, using IEEE C37.118-2011
# Output format is set to None to use native phasor formats. RTPA_Core can automatically converts to Polar or Rectangular floating point value
pdc_buffer.connect("127.0.0.1", port=8900, id_code=235,version="v1", output_format='FloatPolar')

pmu_list = pdc_buffer.list_pmus()
pmu_list

[('SHELBY', 2)]

## Filtering PMUs

You can filter the number of PMUs that you wish to accumulate real-time data from. 
You can also connect to a single PMU by reinitializing the PDCBuffer above with the id_code of the desired PMU.

Note: If you are using openPDC, you can skip this section.

In [ ]:
# You can filter to a specific set of PMUs by using their id_codes or stream_ids based on list_pmus above.
if len(pmu_list) > 1:
    pdc_buffer.set_pmu_filter(id_codes=[1001, 1002])

## Start PDC Stream
Each call to `pdc_buffer.get_data()` will return the latest set of data in the buffer.


In [23]:
# Start the stream. Each call to pdc_buffer.get_data() will give data in the buffer.
# Wait a bit for the buffer to fill.
pdc_buffer.start_stream()

## Real-Time Data Access
The section below demonstrates real time data access to the filtered and formatted pmu data.

The data is returned as a dataframe so you can filter columns and rows like you would any other dataframe.

Note that the data is live and only holds a 2-minute window of data (configurable). If you wish to save the data, you can call `df.to_csv(/path/to/file.csv)` or another save method provided by pandas or polars.

In [24]:
# Data is returned as an arrow record batch using pyarrow.
sleep(10)
record_batch = pdc_buffer.get_data()
df = record_batch.to_pandas()
df.sort_values("DATETIME")
len(df)
df.tail(10)

,DATETIME,SHELBY_2_FREQ_DEVIATION (mHz),SHELBY_2_DFREQ (ROCOF),SHELBY_2_Digital1 (Digital),SHELBY_2_500 kV Bus 1 +SV_magnitude (V),SHELBY_2_500 kV Bus 1 +SV_angle (radians),SHELBY_2_500 kV Bus 2 +SV_magnitude (V),SHELBY_2_500 kV Bus 2 +SV_angle (radians),SHELBY_2_Cordova +SI_magnitude (A),SHELBY_2_Cordova +SI_angle (radians),SHELBY_2_Dell +SI_magnitude (A),SHELBY_2_Dell +SI_angle (radians),SHELBY_2_Lagoon Creek +SI_magnitude (A),SHELBY_2_Lagoon Creek +SI_angle (radians)
356,2025-06-03 03:43:57.000000000,59.962002,0.33,0,299844.46875,0.795056,298851.40625,0.795496,243.557190,-2.513128,522.828857,0.792827,NaN,NaN
357,2025-06-03 03:43:57.033333363,59.965000,-0.30,0,299796.12500,0.787260,298802.15625,0.787792,241.974213,-2.519584,520.486206,0.788383,NaN,NaN
358,2025-06-03 03:43:57.066666666,59.964001,-0.21,0,299800.87500,0.779463,298788.34375,0.779910,241.548187,-2.522048,520.096802,0.783158,NaN,NaN
359,2025-06-03 03:43:57.100000029,59.965000,-0.08,0,299821.06250,0.772365,298824.81250,0.772847,243.429123,-2.529388,522.835205,0.776483,NaN,NaN
360,2025-06-03 03:43:57.133333333,59.966000,0.00,0,299910.43750,0.764981,298895.37500,0.765495,246.711685,-2.549473,526.905396,0.757382,NaN,NaN
361,2025-06-03 03:43:57.166666696,59.962002,0.38,0,299885.56250,0.756542,298886.15625,0.757088,247.320023,-2.556465,528.179504,0.750828,NaN,NaN
362,2025-06-03 03:43:57.200000000,59.965000,-1.09,0,299977.03125,0.750616,298994.71875,0.751086,252.008362,-2.565278,535.820862,0.738987,NaN,NaN
363,2025-06-03 03:43:57.233333363,59.950001,-0.45,0,299772.31250,0.740052,298770.37500,0.740542,245.406876,-2.565593,525.291199,0.740277,NaN,NaN
364,2025-06-03 03:43:57.266666666,59.966999,0.32,0,299825.15625,0.731788,298822.00000,0.732251,240.541718,-2.568792,518.342896,0.738172,NaN,NaN
365,2025-06-03 03:43:57.300000029,59.964001,0.12,0,299817.12500,0.726075,298813.21875,0.726519,246.042984,-2.572580,526.706665,0.730800,NaN,NaN


## Inspecting Raw Data
In some rare cases, you may need to inspect the raw data to debug some values. This is the case for the openPDC Lagoon Creek pmu which sends back Null data. 
If you are running this notebook using the OpenPDC server, you should notice a two instances of "FF C0 00 00" next to each other. These are the NaN values being parsed even though the GUI is showing actual values. 

In [ ]:
import binascii
buffer_sample = pdc_buffer.get_raw_sample()

# Display the raw buffer
print("Raw buffer:")
print(buffer_sample)

# Convert buffer to hex representation with spaces between bytes and 8 bytes per row
hex_bytes = binascii.hexlify(buffer_sample).decode('utf-8')
formatted_hex = ''
for i in range(0, len(hex_bytes), 2):
    # Add a byte (2 hex characters)
    formatted_hex += hex_bytes[i:i+2] + ' '

    # Add a newline after every 32 bytes
    if (i+2) % 64 == 0 and i > 0:
        formatted_hex += '\n'

print("\nHex representation (32 bytes per row):")
print(formatted_hex)

Raw buffer:
b'\xaa\x01\x00D\x00\xebh>o\x84\x00\xbb\xbb\xbb\x80\x00H\x92@\xf6\xbff\xeb\x08H\x91\xc1\xb7\xbff\xcb`C_\xe3\x8a@\x05\xb7\xa5C\xf6\xa9\x0e\xbfb\x18\x83\xff\xc0\x00\x00\xff\xc0\x00\x00Bo\xdd/?(\xf5\xc3\x00\x00D9'

Hex representation (8 bytes per row):
aa 01 00 44 00 eb 68 3e 6f 84 00 bb bb bb 80 00 48 92 40 f6 bf 66 eb 08 48 91 c1 b7 bf 66 cb 60 
43 5f e3 8a 40 05 b7 a5 43 f6 a9 0e bf 62 18 83 ff c0 00 00 ff c0 00 00 42 6f dd 2f 3f 28 f5 c3 
00 00 44 39 


## Cleaning Up
When you are finished with your analysis, you can shut down the pdc_buffer.

In [6]:
pdc_buffer.stop_stream()